In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib widget

In [2]:
from numpy.testing import assert_array_equal

import logging
logging.basicConfig(level=logging.INFO)

In [3]:
import atomlib
from atomlib.visualize import show_atoms_2d, show_atoms_3d
from atomlib.defect import disloc_loop_z, disloc_poly_z, disloc_edge

In [4]:
cell_prim = atomlib.make.wurtzite('AlN', 3.13, 5.02, cell='prim')
cell_ortho = atomlib.make.wurtzite('AlN', 3.13, 5.02, cell='ortho')

show_atoms_3d(cell_prim);

In [5]:
from atomlib.cell import plane_to_zone, reduce_vec

assert_array_equal(plane_to_zone(cell_prim.metric, [1., 0., 0.]), [2., 1., 0.])
assert_array_equal(reduce_vec([4., 2., 0.]), [2, 1, 0])

In [6]:
# make supercell

cell = cell_ortho.repeat((8, 5, 5)).explode()
show_atoms_3d(cell);

In [7]:
# add edge dislocation to supercell

edge = disloc_edge(cell, center=[0., 12., 12.], b=[0., 0., 5.02/2.], t=[1., 0., 0.], cut='add')
show_atoms_3d(edge);

In [8]:
# add vacancies to supercell

vacant = cell.with_occupancy(1 - 1000. / len(cell)).apply_occupancy()  # average 1000 vacancies
print(f"Removed {len(cell) - len(vacant)} atoms")

vacant.write('vacancies2.xsf')

In [9]:
from atomlib.transform import AffineTransform3D
import polars

faults = cell.transform_atoms(AffineTransform3D.translate(0., 5.421/3, 0.), (cell.z() > 11.) & (cell.z() < 26.), frame='local')
faults = faults.wrap()
show_atoms_2d(faults.crop_atoms(x_min=16., x_max=30.), zone=[1,0,0]);

faults.write('fault_pair.xsf')

In [11]:
# weird hourglass dislocation shape

poly = [[-8., -8.], [8., -8.], [-8., 8.], [8., 8.]]
loop = disloc_poly_z(cell, [0., 5.421/3, 5.02/2], poly, center=cell.box_size / 2.)

show_atoms_3d(loop.crop_atoms(x_min=5., x_max=20.));

In [11]:
# circular extrinsic dislocation loop
loop = disloc_loop_z(cell, center=cell.box_size / 2., b=[0., 5.421/3, 5.02/2.,], loop_r=8.)

show_atoms_3d(loop);